In [1]:
#Instalar librerías
%pip install ndlib-viz
#%pip install networkx numpy matplotlib ndlib   

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement ndlib-viz (from versions: none)
ERROR: No matching distribution found for ndlib-viz


In [16]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import scienceplots

import ndlib.models.epidemics as ep
from ndlib.models.ModelConfig import Configuration

# --- 1. Parámetros y Creación de la Red (SIN CAMBIOS) ---
num_nodos = 1000
m_param = 1
seed_global = 42
g = nx.barabasi_albert_graph(num_nodos, m_param, seed=seed_global)
pos = nx.spring_layout(g, seed=seed_global, k=0.6 / np.sqrt(num_nodos), iterations=50)

# --- 2. Configuración del Modelo (SIN CAMBIOS) ---
# Modelo de difusión: ProfileThresholdModel
model = ep.ProfileThresholdModel(g)

# Configuración de parámetros del modelo
config = Configuration()
config.add_model_parameter('beta', 0.8)                # beta: probabilidad de que un nodo intente influenciar a sus vecinos en cada iteración
config.add_model_parameter('threshold', 0.5)           # threshold: umbral mínimo de influencia acumulada para que un nodo cambie de estado
config.add_model_parameter("fraction_infected", 0.25)  # fraction_infected: fracción inicial de nodos que comienzan como infectados

model.set_initial_status(config)

# --- 3. Simulación (SIN CAMBIOS) ---
print("Ejecutando la simulación por 15 iteraciones...")
iterations = model.iteration_bunch(15)
print("Simulación completada.")

# --- 4. Pre-procesamiento de la Historia para la Animación (SIN CAMBIOS) ---
print("\nPre-procesando historial para una animación robusta...")
initial_status = iterations[0].get('status', {})
history_full_status = [initial_status]
last_full_status = initial_status.copy()
for i in range(1, len(iterations)):
    current_full_status = last_full_status.copy()
    delta_status = iterations[i].get('status', {})
    current_full_status.update(delta_status)
    history_full_status.append(current_full_status)
    last_full_status = current_full_status
print("Historial reconstruido con éxito.")

# --- 5. Visualización ---
color_map = {0: '#DEDEEE', 1: '#201F3D'}

# --- Gráfica de Tendencias (CORREGIDA) ---
print("\nGenerando gráfica de tendencias con Scienceplots...")
trends = model.build_trends(iterations)
plt.style.use(['science', 'ieee'])
plt.figure(figsize=(2.5, 2.5))

# <-- CAMBIO AQUÍ: Accedemos directamente a la lista de infectados (clave 1)
infected_trend = trends[0]['trends']['node_count'][1]

plt.plot(infected_trend, color='#201F3D', label='Infectados')
plt.title("Tendencia de la Difusión")
plt.xlabel("Iteración")
plt.ylabel("Número de Nodos")
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.savefig("tendencia_difusion_science.svg", format="svg", bbox_inches='tight')
print("  - Gráfica de tendencias guardada como 'tendencia_difusion_science.svg'")
plt.show()

# --- Gráfica de la Red Final (SIN CAMBIOS) ---
plt.style.use(['science', 'ieee'])
fig, ax = plt.subplots(figsize=(3.5, 3.5))
final_status_corrected = history_full_status[-1]
node_colors = [color_map[final_status_corrected.get(node, 0)] for node in g.nodes()]
nx.draw_networkx_nodes(g, pos, node_color=node_colors, node_size=10, linewidths=0.1, ax=ax)
nx.draw_networkx_edges(g, pos, edge_color="#A7A7A7", alpha=0.5, width=0.3, ax=ax)
ax.set_title("Estado Final de la Difusión")
ax.axis('off')
plt.savefig("red_final_difusion_science.svg", format="svg", bbox_inches='tight')
print("  - Gráfica de red final guardada como 'red_final_difusion_science.svg'")
plt.show()


# --- Animación (SIN CAMBIOS) ---
print("Generando animación final (puede tardar un momento)...")
plt.style.use(['science', 'ieee'])
fig_anim, ax_anim = plt.subplots(figsize=(3.5, 3.5))

def update_frame(frame_number):
    ax_anim.clear()
    status_for_this_frame = history_full_status[frame_number]
    current_colors = [color_map[status_for_this_frame.get(node, 0)] for node in g.nodes()]
    nx.draw_networkx_nodes(g, pos, node_color=current_colors, node_size=10, linewidths=0.1, ax=ax_anim)
    nx.draw_networkx_edges(g, pos, edge_color="#A7A7A7", alpha=0.5, width=0.3, ax=ax_anim)
    ax_anim.set_title(f"Difusión - Iteración {frame_number}")
    ax_anim.axis('off')

num_frames = len(history_full_status)
anim = FuncAnimation(fig_anim, update_frame, frames=num_frames, interval=200)
anim.save('animacion_difusion_science.gif', writer='pillow', fps=5)
plt.close(fig_anim)
print("  - Animación guardada como 'animacion_difusion_science.gif'")

print("\n¡Proceso finalizado con éxito!")

Ejecutando la simulación por 15 iteraciones...
Simulación completada.

Pre-procesando historial para una animación robusta...
Historial reconstruido con éxito.

Generando gráfica de tendencias con Scienceplots...
  - Gráfica de tendencias guardada como 'tendencia_difusion_science.svg'


C:\Users\Trisolaris\AppData\Local\Temp\ipykernel_19532\2003384145.py:67: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


  - Gráfica de red final guardada como 'red_final_difusion_science.svg'
Generando animación final (puede tardar un momento)...


C:\Users\Trisolaris\AppData\Local\Temp\ipykernel_19532\2003384145.py:80: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\Trisolaris\AppData\Local\Temp\ipykernel_19532\2003384145.py:86: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig_anim, ax_anim = plt.subplots(figsize=(3.5, 3.5))


  - Animación guardada como 'animacion_difusion_science.gif'

¡Proceso finalizado con éxito!
